In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
myFile = pd.read_csv('D:/project/550_Pattern/Project_1/550-p1-cset-krk-1.csv')
myFile
myFrame=pd.DataFrame(myFile.values, columns = ["White-King-File", "White-King-rank", "White-Rook-file", "White-Rook-rank","Black-King-file","Black-King-rank","Class"])
myFrame

,White-King-File,White-King-rank,White-Rook-file,White-Rook-rank,Black-King-file,Black-King-rank,Class
0,c,1,e,2,d,3,draw
1,b,1,d,7,c,7,draw
2,d,2,g,1,g,2,draw
3,d,4,g,6,f,6,draw
4,c,1,b,5,a,5,draw
...,...,...,...,...,...,...,...
214,d,3,d,7,a,7,nine
215,b,1,b,4,g,1,nine
216,d,4,e,8,g,2,nine
217,c,2,d,5,f,1,nine


In [3]:
myFrame.shape[0]

219

In [4]:
# Split into training set:60%, holdout set:20%, Validation set: 20%

training_set = myFrame.sample(frac=0.6)
remaining_set = myFrame.drop(training_set.index)
holdout_set = remaining_set.sample(frac=0.5)
validation_set = remaining_set.drop(holdout_set.index)

training_set.reset_index(drop=True, inplace=True)
holdout_set.reset_index(drop=True, inplace=True)
validation_set.reset_index(drop=True, inplace=True)

In [5]:
training_set

,White-King-File,White-King-rank,White-Rook-file,White-Rook-rank,Black-King-file,Black-King-rank,Class
0,c,3,a,5,b,1,one
1,c,2,d,7,a,1,one
2,c,2,g,5,a,4,seven
3,d,3,c,2,a,2,six
4,c,1,b,5,a,3,eight
...,...,...,...,...,...,...,...
126,d,3,d,5,h,2,eight
127,c,3,e,4,c,1,two
128,c,1,e,2,d,3,draw
129,d,3,e,3,d,1,four


In [87]:
holdout_set.shape

(44, 7)

In [7]:
validation_set.shape

(44, 7)

In [8]:
def calculate_entropy(df_label):
    classes,class_counts = np.unique(df_label,return_counts = True)
    entropy_value = np.sum([(-class_counts[i]/np.sum(class_counts))*np.log2(class_counts[i]/np.sum(class_counts)) 
                        for i in range(len(classes))])
    return entropy_value

In [9]:
# dataset = training_set
# feature = Feature or attribute of the dataset, i.e column name (training_set.columns[:-1]) E.g White-King-File
# label = Name of Class label colums, i.e. 'Class'

def calculate_information_gain(dataset,feature,label): 
    # Calculate the dataset entropy
    dataset_entropy = calculate_entropy(dataset[label])   
    values,feat_counts= np.unique(dataset[feature],return_counts=True)
    
    # Calculate the weighted feature entropy                                # Call the calculate_entropy function
    weighted_feature_entropy = np.sum([(feat_counts[i]/np.sum(feat_counts))*calculate_entropy(dataset.where(dataset[feature]
                              ==values[i]).dropna()[label]) for i in range(len(values))])    
    feature_info_gain = dataset_entropy - weighted_feature_entropy    
    return feature_info_gain

In [10]:
class DNode:
    def __init__(self, attribute, values):
        self.attribute = attribute
        self.values = values
        self.children = {}
    

In [11]:
class LeafNode:
    def __init__(self,value):
        self.value = value
        

In [12]:
def copy_list(l):
    return l[:]

In [13]:
attributes = training_set.columns[:-1].tolist()
class_label = 'Class'

In [14]:
def select_attribute(dataset, attributes, class_label):
    
    gains = [calculate_information_gain(dataset, attribute, class_label) for attribute in attributes]
#     print("gains",gains)
    maxGainIndex = np.argmax(gains)
    maxGainValue = gains[maxGainIndex]
#     print("maxGainIndex",maxGainIndex)
#     print("maxGainValue",maxGainValue)
#     print("highGainAttribute:",attributes[maxGainIndex])
    
    
    
    if maxGainValue == 0.0:
        value = np.unique(dataset[class_label])[0]
#         print('value',value)
        return LeafNode(value)
    
    else:
        newAttribute = attributes[maxGainIndex]
        values = np.unique(dataset[newAttribute])
#         print("newAttribute:",newAttribute)
#         print("values:",values)
        return DNode(newAttribute, values)
        

In [15]:
def create_dtree(dataset, attributes, class_label):        
    node = select_attribute(dataset, attributes, class_label)
    
    if isinstance(node, DNode):                        
        newAttributes = copy_list(attributes)
        newAttributes.remove(node.attribute)        
#         print('newAttributes', newAttributes)
#         print('node.values',node.values)
        for v in node.values:
#             print("v:",v)
            newDataSet = dataset.where(dataset[node.attribute] == v).dropna()               
            node.children[v] = create_dtree(newDataSet, newAttributes, class_label)
    
    return node

In [16]:
rootNode = create_dtree(training_set, copy_list(attributes), class_label)

In [17]:
def check_treeRec(node):    
    if isinstance(node, LeafNode):        
        print('leaf node value:', node.value)
        return 

    for k,v in node.children.items():
        check_treeRec(v)
    
    return

In [74]:
def predict(node, row, correctAns):
    if isinstance(node, LeafNode):                
        return node.value == correctAns
#     print('Attr:',node.attribute,", value:", row[node.attribute])
    if row[node.attribute] not in node.children:
        return False
    return predict(node.children[row[node.attribute]], row, correctAns)
    

In [127]:
def calculate_prediction(rootNode, dataset):
    count = 0
    miscount = 0
    misclassed_rows = pd.DataFrame(columns = ["White-King-File", "White-King-rank", "White-Rook-file", "White-Rook-rank","Black-King-file","Black-King-rank","Class"])
    for i in range(dataset.shape[0]):
        ans = predict(rootNode, dataset.loc[i], dataset.loc[i][-1])
        if ans:
            count+=1
        else:
            miscount+=1
            misclassed_rows = misclassed_rows.append(dataset.loc[i], ignore_index = True)
            
    #     print(ans)
    print("miscount:",miscount,"count:",count,", accuracy = ", ((count*100)/dataset.shape[0]),"%")
    return misclassed_rows

In [128]:
misclassed_set = calculate_prediction(rootNode, holdout_set)

miscount: 33 count: 11 , accuracy =  25.0 %


In [130]:
miss_repeated = pd.concat([misclassed_set]*3, ignore_index=True)

In [131]:
miss_repeated

,White-King-File,White-King-rank,White-Rook-file,White-Rook-rank,Black-King-file,Black-King-rank,Class
0,c,2,d,5,f,1,nine
1,d,3,a,3,b,1,five
2,d,1,g,3,h,4,draw
3,d,1,e,4,a,3,five
4,c,1,g,6,h,2,nine
...,...,...,...,...,...,...,...
94,d,2,f,7,a,1,four
95,d,4,d,2,f,1,five
96,c,1,b,8,a,6,eight
97,c,3,f,6,c,1,two


In [136]:
collected_set = miss_repeated.append(training_set, ignore_index = True)

In [137]:
collected_set

,White-King-File,White-King-rank,White-Rook-file,White-Rook-rank,Black-King-file,Black-King-rank,Class
0,c,2,d,5,f,1,nine
1,d,3,a,3,b,1,five
2,d,1,g,3,h,4,draw
3,d,1,e,4,a,3,five
4,c,1,g,6,h,2,nine
...,...,...,...,...,...,...,...
225,d,3,d,5,h,2,eight
226,c,3,e,4,c,1,two
227,c,1,e,2,d,3,draw
228,d,3,e,3,d,1,four


In [155]:
training_set2 = collected_set.sample(training_set.shape[0], replace=True)

In [156]:
training_set2.reset_index(drop=True, inplace=True)
training_set2

,White-King-File,White-King-rank,White-Rook-file,White-Rook-rank,Black-King-file,Black-King-rank,Class
0,d,1,g,3,h,4,draw
1,d,1,d,4,h,1,six
2,c,1,g,2,a,1,draw
3,d,1,g,3,h,4,draw
4,d,3,a,5,h,4,eight
...,...,...,...,...,...,...,...
126,d,4,g,3,h,5,seven
127,d,4,e,4,f,1,eight
128,c,1,c,3,a,1,two
129,c,2,b,3,a,2,one


In [157]:
dtree2_node = create_dtree(training_set2, copy_list(attributes), class_label)

In [162]:
misclassed_set = calculate_prediction(dtree2_node, holdout_set)

miscount: 15 count: 29 , accuracy =  65.9090909090909 %
